In [138]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from pandas import read_csv
from sklearn.utils import shuffle
from datetime import datetime

In [139]:
df_einfach = pd.read_pickle('einfach.pkl') 
df_schwer = pd.read_pickle('schwer.pkl') 

df_data = pd.concat([df_einfach, df_schwer])
df = shuffle(df_data)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [140]:
# new sessionId 
import random
from random import choice
import collections

array_of_ids_einfach = df_einfach['session'].value_counts().index.tolist()
array_of_ids_schwer = df_schwer['session'].value_counts().index.tolist()
numbers_schwer = []

random_numbers_ids_einfach = random.sample(range(101,1000), len(array_of_ids_einfach))
for i in range(len(random_numbers_ids_einfach)):
    random_numbers_ids_schwer = random.choice([i for i in range(101, 1000) if i not in random_numbers_ids_einfach])
    numbers_schwer.append(random_numbers_ids_schwer)

numbers = random_numbers_ids_einfach + numbers_schwer

df_einfach['session'] = df_einfach['session'].replace(array_of_ids_einfach, random_numbers_ids_einfach)
df_schwer['session'] = df_schwer['session'].replace(array_of_ids_schwer, numbers_schwer)

print([item for item, count in collections.Counter(numbers).items() if count > 1])

[]


In [141]:
df_replaced_together = pd.concat([df_einfach, df_schwer])
df_replaced_together['siteName'] = df_replaced_together['siteName'].replace('TestEinfach', 0)
df_replaced_together['siteName'] = df_replaced_together['siteName'].replace('TestSchwer', 1)

# Hier kommt DataFrame sortiert nach Session

In [142]:
df_replaced_together = df_replaced_together.sort_values(by=['session','date'], ascending=[False, True])

In [143]:
df_replaced_together['real_date'] = df_replaced_together['date'].astype('datetime64[ns]')
#df_replaced_together['real_date'] = (df_replaced_together['real_date'] - df_replaced_together['real_date'].min())  / np.timedelta64(1,'D')
df_replaced_together = df_replaced_together.drop(['date', 'clientX', 'clientY', 'sitename', 'key', 'timeStamp'], axis=1)
df_replaced_together = df_replaced_together.loc[df_replaced_together["type"] !="scroll"]

In [144]:
hey = pd.DataFrame(numbers, columns=["session"])
arr1 = []
arr2 = []
arr3 = []
arr4 = []
arr5 = []
arr6 = []
arr7 = []
arr8 = []
arrDate = []
arrHeight = []
arrWidth = []
arrSiteName = []

for i in range(len(numbers)):
    new = df_replaced_together.loc[df_replaced_together['session'] == numbers[i]]
    #get first and last entry of new dataframe
    #first = new['real_date'].iloc[0]
    date = (new['real_date'].iloc[-1] - (new['real_date'].iloc[0])).total_seconds()
    arrDate.append(date)
    height = new['height'].iloc[0]
    arrHeight.append(height)
    width = new['width'].iloc[0]
    arrWidth.append(width)
    siteName = new['siteName'].iloc[0]
    arrSiteName.append(siteName)
    #print(new)
    #print(date)
    # last = new.iloc[-1]
    new2 = (len(new.loc[new['type']=='mouseover']))
    arr2.append(new2)
    new3 = (len(new.loc[new['type'] =='mouseout']))
    arr3.append(new3)
    new4 = (len(new.loc[new['type'] =='click']))
    arr4.append(new4)
    new5 = (len(new.loc[new['type'] =='blur']))
    arr5.append(new5)
    new6 = (len(new.loc[new['type'] =='focus']))
    arr6.append(new6)
    new7 = (len(new.loc[new['type'] =='keydown']))
    arr7.append(new7)

hey['count_mouseover'] = arr2
hey['count_mouseout'] = arr3
hey['count_click'] = arr4
hey['count_blur'] = arr5
hey['count_focus'] = arr6
hey['count_keydown'] = arr7
hey['duration'] = arrDate
hey['height'] = arrHeight
hey['width'] = arrWidth
hey['siteName'] = arrSiteName

# for i in range(len(types)):
#     hey['type_' + str(types[i])] = arr



#hey['duration'] = df_replaced_together['date']


In [145]:
hey = hey.loc[hey['duration'] < 2000]

In [146]:
hey.head()

,session,count_mouseover,count_mouseout,count_click,count_blur,count_focus,count_keydown,duration,height,width,siteName
0,674,6,6,0,7,7,0,45.545,736,393,0
1,129,6,6,0,6,6,0,54.128,572,360,0
2,474,6,6,0,7,7,0,18.090,775,412,0
3,741,6,6,0,7,7,0,27.660,827,412,0
4,208,6,6,0,6,6,0,36.191,765,412,0


In [148]:
hey.to_pickle('dataSessions.pkl')